### shortterm_model_regressor-v6

predicting target: end_rank = start_rank + change(old target)

base: 14./stage_model_regressor_withneighbor-newfeatures

prediction models of chg_of_rank_in_stage on stage dataset

data format:
    target , eventid ,    car_number,    stageid,     features...

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pickle

In [2]:
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.linear_model.ridge import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.svm.classes import SVR
from sklearn.utils import shuffle
from sklearn import metrics
import xgboost as xgb

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.ridge module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning:

In [3]:
# bulid regression model
regressors = ['currank','avgrank','dice','lasso','ridge','rf','svr','xgb']
train_x, train_y, test_x, test_y = None, None, None, None

def get_regressor(regressor = 'lr'):
    if regressor == "lasso":
        clf = LassoCV(cv=5, random_state=0)
    elif regressor == "ridge":
        clf = RidgeCV(alphas=np.logspace(-6, 6, 13))
    elif regressor == "rf":
        clf = RandomForestRegressor(n_estimators=100)
    elif regressor == 'svr':
        clf = SVR(kernel='rbf')
    elif regressor == 'xgb':
        clf = xgb.XGBRegressor(objective="reg:linear", random_state=42, max_depth=3)
    elif regressor == 'dice':
        clf = RandomDice('1234')
    elif regressor == 'currank':
        clf = CurRank()
    elif regressor == 'avgrank':
        clf = AverageRank()        
    else:
        clf = None
        
    return clf


class CurRank():
    """
    predict with current rank
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = [0 for x in range(test_x.shape[0])]
        return np.array(pred_y)
    
class AverageRank():
    """
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            #13, change_in_rank_all
            pred_y.append(x[13])
        #pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in pred_y])
        pred_y_avg = pred_y
        return np.array(pred_y_avg)   

class RandomDice():
    """
    a random dice model
    """
    def __init__(self, seed='1234'):
        self.dist = []
        self.val = []
        random.seed(seed)
    
    def fit(self, x, y):
        total = y.shape[0]
        yval = set(y)
        
        ratio = 0.
        for val in yval:
            self.val.append(val)
            ratio += np.sum(y==val)*1.0 / total
            self.dist.append(ratio)
            
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            dice = random.random()
            #search in self.dist
            find_idx = -1
            for idx, ratio in enumerate(self.dist):
                if dice <= ratio:
                    find_idx = idx
                    break
            
            #or the last one match
            pred_y.append(self.val[find_idx])
            
        return np.array(pred_y)

def evaluate(test_y, pred_y):
    mae = metrics.mean_absolute_error(test_y, pred_y) 
    rmse = math.sqrt(metrics.mean_squared_error(test_y, pred_y))
    r2 = metrics.r2_score(test_y, pred_y)
    print('rmse=%.2f, mae=%.2f, r2=%.2f'%(rmse, mae, r2))
    return mae,rmse, r2
    
#
#features
#    cols=[Myidx, 'target','eventid','car_number','stageid',
#             'firststage','pit_in_caution','start_position',
#             'start_rank','start_rank_ratio','top_pack','bottom_pack',
#             'average_rank','average_rank_all',
#             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all']    
def split_by_eventidx(stagedata, eventid):
    """
    split by eventid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['eventid'].isin(train_events)].to_numpy()
    test  = stagedata[stagedata['eventid'].isin(test_events)].to_numpy()

    #2:car_number
    train_x = train[:,2:]
    #train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    train_y = train[:,1]
    test_x = test[:,2:]
    #test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    test_y = test[:,1]
    
    #change target to endrank
    #train_y = train_y + train[:,8] 
    #test_y = test_y + test[:,8]
    
    train = stagedata[stagedata['eventid'].isin(train_events)]
    test  = stagedata[stagedata['eventid'].isin(test_events)]
    
    return train, test, train_x, train_y, test_x, test_y

def split_by_eventid(stagedata, train_events, test_events, startid=2):
    """
    split by eventid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['eventid'].isin(train_events)].to_numpy()
    test  = stagedata[stagedata['eventid'].isin(test_events)].to_numpy()

    #2:car_number
    train_x = train[:,startid:]
    #train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    train_y = train[:,1]
    test_x = test[:,startid:]
    #test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    test_y = test[:,1]
    
    #change target to endrank
    #train_y = train_y + train[:,8] 
    #test_y = test_y + test[:,8]
    
    train = stagedata[stagedata['eventid'].isin(train_events)]
    test  = stagedata[stagedata['eventid'].isin(test_events)]
    
    return train, test, train_x, train_y, test_x, test_y

def split_by_stageid(stagedata, stageid):
    """
    split by stageid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['stageid'] <= stageid].to_numpy()
    test  = stagedata[stagedata['stageid'] > stageid].to_numpy()

    train_x = train[:,2:]
    #train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    train_y = train[:,1]
    test_x = test[:,2:]
    #test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    test_y = test[:,1]
    
    return train, test, train_x, train_y, test_x, test_y


def regressor_model(name='svr'):
    ### test learning models
    print('[*] predict with %s model'%name)
    clf = get_regressor(name)
    clf.fit(train_x, train_y)

    pred_y = clf.predict(test_x)
    
    #int only
    #pred_y = pred_y.astype(int)

    
    score = evaluate(test_y, pred_y)
    return score, pred_y

In [4]:
def do_rerank(dfout, col=4):
    """
    output of prediction of target can be float
    ['carno','startlap', 'startrank','endrank']
    resort the endrank globally
    
    """
    
    #df = dfout.sort_values(by=['startlap','carno'])
    print('rerank...')
    laps = set(dfout.startlap.values)
    
    dfs = []
    for lap in laps:
        df = dfout[dfout['startlap']==lap].to_numpy()
        
        #print('in',df)
        
        idx = np.argsort(df[:,col], axis=0)
        true_rank = np.argsort(idx, axis=0)
    
        df[:,col] = true_rank
        
        #print('out',df)
        if len(dfs) == 0:
            dfs = df
        else:
            dfs = np.vstack((dfs, df))
        #dfs.append(df)
        #np.vstack(df)
        
    #dfret = pd.concat(dfs)
    #data = np.array(dfs)
    dfret = pd.DataFrame(dfs, columns =['carno','startlap', 'startrank','endrank','pred_endrank'])
    
    return dfret

def build_df(testdf, pred_y, dorerank=True):
    """
    build a standard stint prediction result:
    carno	startlap	startrank	endrank	diff	sign	pred_endrank	pred_diff	pred_sign
    """
    
    print('build_df: len testdf=%d, len of pred_y=%d'%(len(testdf), len(pred_y)))
    
    test = testdf[['car_number','start_lap','start_rank','target']].values
    
    # todo: fix
    # start_lap already been 0-index number
    # current code in Model_compare_ranknet_vs_others-v4-all.ipynb::long_predict_bymloutput
    # follows this 'wrong' index
    test[:,1] = test[:,1]-1
    
    test[:,2] = test[:,2]-1
    test[:,3] = test[:,2] + test[:,3]
    dfout = pd.DataFrame(test, columns =['carno','startlap', 'startrank','endrank'])
    

    # add predictions
    dfout['pred_endrank'] = pred_y +  dfout['startrank']
    
    if dorerank:
        dfout = do_rerank(dfout,col=4)    
    
    dfout['diff'] = dfout['endrank'] - dfout['startrank']
    signVec = dfout['diff'].values.copy()
    for idx in range(len(signVec)):
        sign = 0
        if signVec[idx] > 0:
            sign = 1
        elif signVec[idx] < 0:
            sign = -1
        signVec[idx] = sign
    dfout['sign'] = signVec


    #dfout['pred_diff'] = pred_y 
    dfout['pred_diff'] = dfout['pred_endrank'] - dfout['startrank']
    signVec = dfout['pred_diff'].values.copy()
    for idx in range(len(signVec)):
        sign = 0
        if signVec[idx] > 0:
            sign = 1
        elif signVec[idx] < 0:
            sign = -1
        signVec[idx] = sign
    dfout['pred_sign'] = signVec
    return dfout
    

def test_cv():
    global train_x, train_y, test_x, test_y
    
    cols = ['runid','trainsize','testsize','testdistribution']
    cols.extend(regressors)
    print('cols:%s'%cols)
    retdf0 = pd.DataFrame([],columns=cols)
    retdf1 = pd.DataFrame([],columns=cols)

    events = set(stagedata['eventid'])

    years = ['2013','2014','2015','2016','2017','2018','2019']
    #events = ['Indy500']
    eventsname = [f'Indy500-{x}' for x in years]
    events_id={key:idx for idx, key in enumerate(eventsname)}
    for eventid in events:
        print('Testset = %s'%eventsname[eventid])

        train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
        test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
        #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
        #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))

        #record
        rec0 = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]
        rec1 = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

        pred_y = [0 for x in range(len(regressors))]
        acc0 = [0 for x in range(len(regressors))]
        acc1 = [0 for x in range(len(regressors))]
        for idx, clf in enumerate(regressors):
            acc, pred_y[idx] = regressor_model(clf)
            acc0[idx] = acc[0]
            acc1[idx] = acc[2]

        rec0.extend(acc0)
        rec1.extend(acc1)
        #print('rec:%s'%rec)

        #new df
        df = pd.DataFrame([rec0],columns=cols)
        retdf0 = pd.concat([retdf0, df])        

        df = pd.DataFrame([rec1],columns=cols)
        retdf1 = pd.concat([retdf1, df])        


    #retdf0.to_csv('regressors_stagedata_splitbyevent%s_rmse.csv'%suffix)
    #retdf1.to_csv('regressors_stagedata_splitbyevent%s_r2.csv'%suffix)
    retdf0.to_csv('crossvalid_stagedata_regressor_%s.csv'%suffix, float_format='%.3f')

    df_event_rmse = retdf0
    df_event_r2 = retdf1
    return df_event_rmse
    
def test_20182019(dorerank=True, startid=2):
    global train_x, train_y, test_x, test_y

    pred_df = {}
    
    ### train 2013-2017
    #load data
    cols = ['runid','trainsize','testsize','testdistribution']
    cols.extend(regressors)
    print('cols:%s'%cols)
    retdf0 = pd.DataFrame([],columns=cols)
    retdf1 = pd.DataFrame([],columns=cols)

    eventsids = set(stagedata['eventid'])

    for event in events:
        test_eventid = events_id[event]
        
        if test_eventid in _train_events:
            continue
        
        print('Testset = %s'% event, test_eventid)

        traindf, testdf, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, 
                                                            _train_events, [test_eventid], startid=startid)
        test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
        #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
        #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))

        #record
        rec0 = [event,train_x.shape[0],test_x.shape[0],test_distribution]
        rec1 = [event,train_x.shape[0],test_x.shape[0],test_distribution]

        pred_y = [0 for x in range(len(regressors))]
        acc0 = [0 for x in range(len(regressors))]
        acc1 = [0 for x in range(len(regressors))]
        for idx, clf in enumerate(regressors):
            acc, pred_y[idx] = regressor_model(clf)
            acc0[idx] = acc[0]
            acc1[idx] = acc[2]

            #build pred df
            if not event in pred_df:
                pred_df[event] = {}
            pred_df[event][clf] = build_df(testdf, pred_y[idx],dorerank=dorerank)


        rec0.extend(acc0)
        rec1.extend(acc1)
        #print('rec:%s'%rec)

        #new df
        df = pd.DataFrame([rec0],columns=cols)
        retdf0 = pd.concat([retdf0, df])            
    
    return retdf0, pred_df   

In [5]:
import pickle
def save_result(dfs, datafile):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [dfs]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

### test oracle with stint_len

In [6]:
_inlap_status = 0
_featureCnt = 9
#
# input data parameters
#
# event -> car#, maxlap
_race_info = {}
# the races have 7 years data 
races = ['Indy500', 'Texas','Iowa','Pocono']
years = ['2013','2014','2015','2016','2017','2018','2019']

events = []
for race in races:
    events.extend([f'{race}-{x}' for x in years])

events.extend(['Phoenix-2018','Gateway-2018','Gateway-2019'])
events_id={key:idx for idx, key in enumerate(events)}

# dataset shared
dataOutputRoot = "data/"
covergap = 1
dbid = f'IndyCar_d{len(events)}_v{_featureCnt}_p{_inlap_status}'
LAPTIME_DATASET = f'{dataOutputRoot}/laptime_rank_timediff_pit-oracle-{dbid}.pickle' 
STAGE_DATASET = f'{dataOutputRoot}/stagedata-{dbid}.pickle' 
PITCOVERED_DATASET = f'{dataOutputRoot}/pitcoveredlaps-{dbid}-g{covergap}.pickle'
PITSTOP_DATASET = f'{dataOutputRoot}/pitstop-{dbid}.csv' 
    
print('Load laptime and stage dataset:',LAPTIME_DATASET, STAGE_DATASET)
with open(LAPTIME_DATASET, 'rb') as f:
    global_carids, laptime_data = pickle.load(f, encoding='latin1') 
with open(STAGE_DATASET, 'rb') as f:
    #stagedata = pickle.load(f, encoding='latin1') 
    stagedata_o, _race_info, _events, _events_id = pickle.load(f, encoding='latin1') 
pitstop_df = pd.read_csv(PITSTOP_DATASET)

#check it
if not _events == events:
    print('Error, events mismatch at:', STAGE_DATASET)
    sys.exit(-1)

#trainrace = 'Indy500','Iowa','Pocono','Texas'
trainrace = 'Indy500'
    
#_train_events = [events_id[x] for x in [f'{trainrace}-{x}' for x in ['2013','2014','2015','2016','2017']]]
#_test_events = [f'{trainrace}-{x}' for x in ['2018','2019']]
if trainrace == 'Pocono':
    # 2014, 2019 is bad
    _train_years = ['2013','2015','2016','2017']
    testevents = [f'{trainrace}-2018']
else:
    _train_years = ['2013','2014','2015','2016','2017']
    testevents = [f'{trainrace}-2018',f'{trainrace}-2019']
    
_train_events = [events_id[x] for x in [f'{trainrace}-{x}' for x in _train_years]]

Load laptime and stage dataset: data//laptime_rank_timediff_pit-oracle-IndyCar_d31_v9_p0.pickle data//stagedata-IndyCar_d31_v9_p0.pickle


In [7]:
#load data
_trim = 0
_include_final = True
_include_stintlen = True
#_include_stintlen = False
include_str = '1' if _include_final else '0'
stint_str = '1' if _include_stintlen else ''


_context_len = 60

In [8]:
#version = f'IndyCar-d{len(events)}'
version = f'IndyCar-d{trainrace}'

for plen in [2,4,6,8]:
#for plen in [2]:

    output_file = f'{dataOutputRoot}/shortterm-IndyCar-d{len(events)}-end{include_str}{stint_str}-t{plen}-c{_context_len}.csv'
    stagedata = pd.read_csv(output_file)
    stagedata.fillna(0, inplace=True)
    #stagedata.info()
    retdf_rerank, preddf_rerank = test_20182019(dorerank=True)
    retdf_norank, preddf_norank = test_20182019(dorerank=False)

    #output_file = f'stage-indy500-2013-2019-end{include_str}{stint_str}-t{_trim}.csv'
    #outfile=f'shortterm-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-normal-t{_trim}.pickle'
    #save_result(preddf, outfile)
    outfile=f'{dataOutputRoot}/shortterm-dfout-mlmodels-{version}-end{include_str}-rerank-t{plen}-c{_context_len}.pickle'
    save_result(preddf_rerank, outfile)
    outfile=f'{dataOutputRoot}/shortterm-dfout-mlmodels-{version}-end{include_str}-norerank-t{plen}-c{_context_len}.pickle'
    save_result(preddf_norank, outfile)


cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lasso', 'ridge', 'rf', 'svr', 'xgb']
Testset = Indy500-2018 5
[*] predict with currank model
rmse=3.25, mae=1.36, r2=-0.00
build_df: len testdf=5340, len of pred_y=5340
rerank...
[*] predict with avgrank model
rmse=17.57, mae=15.58, r2=-28.14
build_df: len testdf=5340, len of pred_y=5340
rerank...
[*] predict with dice model
rmse=4.37, mae=2.25, r2=-0.81
build_df: len testdf=5340, len of pred_y=5340
rerank...


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 47.82735999964643, tolerance: 16.681904147635525
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 86.61634503304958, tolerance: 16.681904147635525
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 93.12000701832585, tolerance: 16.681904147635525
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-package

[*] predict with lasso model
rmse=3.10, mae=1.61, r2=0.09
build_df: len testdf=5340, len of pred_y=5340
rerank...
[*] predict with ridge model
rmse=3.10, mae=1.61, r2=0.09
build_df: len testdf=5340, len of pred_y=5340
rerank...
[*] predict with rf model
rmse=3.38, mae=1.92, r2=-0.08
build_df: len testdf=5340, len of pred_y=5340
rerank...
[*] predict with svr model
rmse=3.25, mae=1.39, r2=0.00
build_df: len testdf=5340, len of pred_y=5340
rerank...
[*] predict with xgb model
[09:40:14] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
rmse=3.62, mae=1.94, r2=-0.23
build_df: len testdf=5340, len of pred_y=5340
rerank...
Testset = Indy500-2019 6
[*] predict with currank model
rmse=3.07, mae=1.18, r2=-0.00
build_df: len testdf=5629, len of pred_y=5629
rerank...
[*] predict with avgrank model
rmse=18.16, mae=16.11, r2=-33.95
build_df: len testdf=5629, len of pred_y=5629
rerank...
[*] predict with dice model
rmse=4.15, mae=2.0

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 47.82735999964643, tolerance: 16.681904147635525
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 86.61634503304958, tolerance: 16.681904147635525
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 93.12000701832585, tolerance: 16.681904147635525
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-package

rmse=2.93, mae=1.45, r2=0.09
build_df: len testdf=5629, len of pred_y=5629
rerank...
[*] predict with ridge model
rmse=2.92, mae=1.43, r2=0.10
build_df: len testdf=5629, len of pred_y=5629
rerank...
[*] predict with rf model
rmse=2.96, mae=1.62, r2=0.07
build_df: len testdf=5629, len of pred_y=5629
rerank...
[*] predict with svr model
rmse=3.07, mae=1.23, r2=0.00
build_df: len testdf=5629, len of pred_y=5629
rerank...
[*] predict with xgb model
[09:43:49] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
rmse=3.04, mae=1.51, r2=0.02
build_df: len testdf=5629, len of pred_y=5629
rerank...
Testset = Texas-2013 7
[*] predict with currank model
rmse=1.40, mae=0.52, r2=0.00
build_df: len testdf=4764, len of pred_y=4764
rerank...
[*] predict with avgrank model
rmse=13.21, mae=11.70, r2=-87.67
build_df: len testdf=4764, len of pred_y=4764
rerank...
[*] predict with dice model
rmse=3.09, mae=1.48, r2=-3.84
build_df: len testdf=4

...

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 87.10716592584504, tolerance: 43.95080792458247
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 85.49322204478085, tolerance: 43.95080792458247
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 83.97078571812017, tolerance: 43.95080792458247
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/s

rmse=2.29, mae=1.70, r2=-0.22
build_df: len testdf=4490, len of pred_y=4490
[*] predict with ridge model
rmse=2.10, mae=1.46, r2=-0.03
build_df: len testdf=4490, len of pred_y=4490
[*] predict with rf model
rmse=4.75, mae=3.79, r2=-4.26
build_df: len testdf=4490, len of pred_y=4490
[*] predict with svr model
rmse=1.99, mae=1.16, r2=0.07
build_df: len testdf=4490, len of pred_y=4490
[*] predict with xgb model
[22:20:00] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
rmse=6.38, mae=5.01, r2=-8.50
build_df: len testdf=4490, len of pred_y=4490
Testset = Gateway-2019 30
[*] predict with currank model
rmse=2.81, mae=1.47, r2=-0.00
build_df: len testdf=4550, len of pred_y=4550
[*] predict with avgrank model
rmse=12.62, mae=11.06, r2=-19.20
build_df: len testdf=4550, len of pred_y=4550
[*] predict with dice model
rmse=5.35, mae=3.42, r2=-2.63
build_df: len testdf=4550, len of pred_y=4550
[*] predict with lasso model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 87.64064061932731, tolerance: 43.95080792458247
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 364.0697397561744, tolerance: 43.95080792458247
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 414.60163228493184, tolerance: 43.95080792458247
  tol, rng, random, positive)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6740.306740051368, tolerance: 51.418678569787915
  positive)


rmse=2.97, mae=2.09, r2=-0.12
build_df: len testdf=4550, len of pred_y=4550
[*] predict with ridge model
rmse=2.84, mae=1.92, r2=-0.02
build_df: len testdf=4550, len of pred_y=4550
[*] predict with rf model
rmse=4.67, mae=3.64, r2=-1.77
build_df: len testdf=4550, len of pred_y=4550
[*] predict with svr model
rmse=2.66, mae=1.60, r2=0.10
build_df: len testdf=4550, len of pred_y=4550
[*] predict with xgb model
[22:23:40] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
rmse=6.30, mae=4.81, r2=-4.04
build_df: len testdf=4550, len of pred_y=4550


### test

In [9]:
sys.exit(0)

NameError: name 'sys' is not defined

In [ ]:
def loaddata(datafile):
    with open(datafile, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        dfout = pickle.load(f, encoding='latin1') 
        return dfout   

In [ ]:
#load data
_trim = 0
_include_final = True
_include_stintlen = True
#_include_stintlen = False
include_str = '1' if _include_final else '0'
stint_str = '1' if _include_stintlen else ''
suffix = f'indy500-2013-2019-end{include_str}{stint_str}-t{_trim}'
plen=2
outfile=f'shortterm-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-rerank-t{plen}.pickle'
preddf_rerank = loaddata(outfile)[0]

In [ ]:
output_file = f'shortterm-indy500-2013-2019-end{include_str}{stint_str}-t{plen}-.csv'
stagedata = pd.read_csv(output_file)
stagedata.fillna(0, inplace=True)


In [ ]:
stagedata.info()

In [ ]:
df = preddf_rerank['2018']['xgb']

In [ ]:
df[df['carno']==12]

In [ ]:
df = preddf_rerank['2018']['svr']

In [ ]:
dfx = df[df['carno']==12]

In [ ]:
dfx[dfx['startlap']>190]

In [ ]:
### test blackhorse car=27
df2018 = preddf_oracle['2018']['rf']
car27 = df2018[df2018['carno']==27]

In [ ]:
car27

In [ ]:
evaluate(car27['pred_diff'].values,car27['diff'].values)

In [ ]:
### test blackhorse car=27
df2018 = preddf_oracle['2018']['xgb']
car27 = df2018[df2018['carno']==12]
car27

In [ ]:
evaluate(car27['pred_diff'].values,car27['diff'].values)

In [ ]:
car27 = df2018
evaluate(car27['pred_diff'].values,car27['diff'].values)

In [ ]:
len(df2018)

In [ ]:
retdf_oracle

In [ ]:
top10 = [12, 20, 9, 27, 28, 22, 29, 1, 6]
car27 = df2018[df2018['carno'].isin(top10)]
evaluate(car27['pred_diff'].values,car27['diff'].values)

In [ ]:
top10 = [12, 20, 9, 27, 28]
car27 = df2018[df2018['carno'].isin(top10)]
evaluate(car27['pred_diff'].values,car27['diff'].values)

In [ ]:
df2018 = preddf['2018']['xgb']
car27 = df2018[df2018['carno'].isin(top10)]
evaluate(car27['pred_diff'].values,car27['diff'].values)

In [ ]:
preddf_rerank['Indy500-2018']